Графики для анализа данных фед ресурса mdm

In [ ]:
# вызов общей функции для создания fig
def create_plot(figCreation, *args, **kwargs):
    x_title = kwargs.get('x_title', None)
    y_title = kwargs.get('y_title', None)
    y_title_second = kwargs.get('y_title_second', None)
    
    x_range = kwargs.get('x_range', None)
    y_range = kwargs.get('y_range', None)
    
    title = kwargs.get('title', None)

    height = kwargs.get('height', 500)
    width = kwargs.get('width', 700)
    autosize = kwargs.get('autosize', False)
        
    fig = figCreation(*args, **kwargs)
    
    if x_title:
        fig.update_xaxes(title_text=x_title)
    if y_title:
        fig.update_yaxes(title_text=y_title)
    
    if y_title_second:
        fig.update_yaxes(title_text=y_title_second, secondary_y=True)
    
    if x_range: 
        fig.update_xaxes(range=x_range)
    if y_range: 
        fig.update_yaxes(range=y_range)
        
    if title:
        fig.update_layout(title=title)
    
    if autosize==False:
        fig.update_layout(autosize=False, width=width, height=height,)
        
    fig.update_layout(modebar_add=['hovercompare'])

    return fig

In [ ]:
# графики
def show_ActiveContracts(target):

    fraction_column = 'Доля {type}'.format(type=target.type)

    name_0 = "{type}=0".format(type=target.type)
    name_1 = "{type}=1".format(type=target.type)

    hist = px.histogram(
        target.data, x='no_contracts_active',  y='Приложение', 
        title="Количество активных договоров", 
        color = 'Метка дефолта', 
        barmode='group',
        histfunc='count'
    )

    share = (pd.pivot_table(target.data, index=['no_contracts_active'],  values='Метка дефолта',   
                aggfunc=lambda x: x.sum()/x.count() if len(x)>0 else 0, fill_value=0)
                .reset_index().rename(columns={'Метка дефолта':fraction_column, 
                                    'no_contracts_active':'Количество активных договоров'})
        )
    
    
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    fig.update_layout(
            updatemenus=[
                dict(
                        buttons=list([
                            dict(
                                args=[{"mode": 'lines+markers'}],
                                label="Hide text",
                                method="restyle"
                            ),
                            dict(
                                args=[{"mode": 'lines+markers+text'}],
                                label="Show text",
                                method="restyle"
                            ),
                        ]),
                    ),
            ]
        )
    

    fig.add_trace(go.Histogram(x=hist.data[0].x, y=hist.data[0].y, name=name_0, texttemplate="%{y}"),  secondary_y=False)
    fig.add_trace(go.Histogram(x=hist.data[1].x, y=hist.data[1].y, name=name_1, texttemplate="%{y}"),  secondary_y=False)
    fig.add_trace(go.Scatter(x=share['Количество активных договоров'], y=share[fraction_column], name=fraction_column, 
                                mode='lines+markers', texttemplate = "%{y:.2f}", textposition = "top center") , secondary_y=True)

    fig.update_layout(title_text="Количество target договоров и доля {type} меток".format(type=target.type))

    # Set y-axes titles
    fig.update_yaxes(title_text="Количество target договоров", secondary_y=False, type='log')
    fig.update_yaxes(title_text="Доля {type} меток".format(type=target.type), secondary_y=True)

    fig.update_xaxes(title_text="Количество активных договоров на момент принятия решения")

    config = dict({'modeBarButtonsToAdd': ['hovercompare']})

    fig.update_layout(autosize=False, width=900, height=500,)
    fig.show('png' if need_svg else '', config=config)

In [1]:
# графики c sigmoid
def show_predictions(target, sigmoid_pred, feature, **kwargs):
    x_range = kwargs.get('x_range', None)
    y_range = kwargs.get('y_range', None)
    
    mask = kwargs.get('mask', None)
    if isinstance(mask, pd.Series):
        df = target.data[mask]
    else:
        df = target.data

    title = kwargs.get('title', feature)
    
    name_0 = "{type}=0".format(type=target.type)
    name_1 = "{type}=1".format(type=target.type)

    hist = px.histogram(
        df, x=feature,  y='Приложение', 
        # title="Количество активных договоров", 
        color = 'Метка дефолта', 
        barmode='group',
        histfunc='count'
    )
    
    default = sigmoid_pred[sigmoid_pred['y_pred']==1]
    not_default = sigmoid_pred[sigmoid_pred['y_pred']!=1]

    fig = make_subplots(specs=[[{"secondary_y": True}]])

  

    fig.add_trace(go.Histogram(x=hist.data[0].x, y=hist.data[0].y, name=name_0, texttemplate="%{y}"),  secondary_y=False)
    fig.add_trace(go.Histogram(x=hist.data[1].x, y=hist.data[1].y, name=name_1, texttemplate="%{y}"),  secondary_y=False)
    fig.add_trace(go.Scatter(x=default['X'], y=default['y_prob'], name='predicted 1', 
                                mode='markers', texttemplate = "%{y:.2f}", textposition = "top center" , 
                                marker=dict(
                                            size=10,
                                            color = 3, 
                                            colorscale='Viridis', # one of plotly colorscales
                                            )
                            ) , secondary_y=True)

    fig.add_trace(go.Scatter(x=not_default['X'], y=not_default['y_prob'], name='predicted 0', 
                                mode='markers', texttemplate = "%{y:.2f}", textposition = "top center" , 
                                marker=dict(
                                            size=10,
                                            color = 0, 
                                            colorscale='Bluered_r', # one of plotly colorscales
                                            )
                            ) , secondary_y=True)

    fig.update_layout(title_text="Количество target договоров и Вероятность дефолта predicted (balanced)".format(type=target.type))

    # Set y-axes titles
    fig.update_yaxes(title_text="Количество target договоров", secondary_y=False, type='log')
    # fig.update_yaxes(title_text="Количество target договоров", secondary_y=False, type='linear')
    fig.update_yaxes(title_text="Вероятность дефолта (balanced)".format(type=target.type), secondary_y=True)

    fig.update_xaxes(title_text=title)

    if x_range: 
        fig.update_xaxes(range=x_range)
    if y_range: 
        fig.update_yaxes(range=y_range)
        
    config = dict({'modeBarButtonsToAdd': ['hovercompare']})

    fig.update_layout(autosize=False, width=700, height=400,)
    fig.show('png' if need_svg else '', config=config)

    # fig.write_html("file.html")
    return fig

In [ ]:
def show_probability(target, sigmoid_pred, feature, **kwargs):

    mask = kwargs.get('mask', None)
    title_text = kwargs.get('title_text', "Количество договоров и Вероятность дефолта в обучающей выборке")
    title = kwargs.get('title', feature)
    show = kwargs.get('show', True)
    
    if isinstance(mask, pd.Series):
        df = target.data[mask]
    else:
        df = target.data

    
    
    name_0 = "{type}=0".format(type=target.type)
    name_1 = "{type}=1".format(type=target.type)

    hist = px.histogram(
        df, x=feature,  y='Приложение', 
        # title="Количество активных договоров", 
        color = 'Метка дефолта', 
        barmode='group',
        histfunc='count'
    )
    
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    fig.add_trace(go.Histogram(x=hist.data[0].x, y=hist.data[0].y, name=name_0, texttemplate="%{y}"),  secondary_y=False)
    fig.add_trace(go.Histogram(x=hist.data[1].x, y=hist.data[1].y, name=name_1, texttemplate="%{y}"),  secondary_y=False)
    fig.add_trace(go.Scatter(x=sigmoid_pred['X'], y=sigmoid_pred['y_prob'], name='probability', 
                                mode='markers', texttemplate = "%{y:.2f}", textposition = "top center" , 
                                marker=dict(
                                            size=10,
                                            color = 3, 
                                            colorscale='Viridis', # one of plotly colorscales
                                            )
                            ) , secondary_y=True)

    fig.update_layout(title_text=title_text)

    # Set y-axes titles
    fig.update_yaxes(title_text="Количество договоров", secondary_y=False, type='log')
    # fig.update_yaxes(title_text="Количество target договоров", secondary_y=False, type='linear')
    fig.update_yaxes(title_text="Вероятность дефолта".format(type=target.type), secondary_y=True)

    fig.update_xaxes(title_text=title)

    config = dict({'modeBarButtonsToAdd': ['hovercompare']})

    fig.update_layout(autosize=False, width=700, height=400,)
    if show:
        fig.show('png' if need_svg else '', config=config)
   
    
    # fig.write_html("file.html")
    return fig

In [1]:
#  печатать распределение количества приложений от значение фичи
def plot_features_bar(df, feature, type='фрод', **kwargs):
    x_range = kwargs.get('x_range', None)
    y_range = kwargs.get('y_range', None)
    show = kwargs.get('show', True)

    tmp = pd.pivot_table(df, index=[feature, 'Метка дефолта'], values='Приложение', aggfunc='count', fill_value=0, margins=True).reset_index()
    
    labels = {"no_contracts_active": "Количество активных договоров", 
              "no_contracts_total": "Количество договоров до дня принятия решения",
              "no_transp_active": "Количество активных договоров только по ТС",
              "no_transport_total": "Количество договоров только с ТС до дня принятия решения",
              "Приложение": "Количество сделок"
             }

    fig = px.bar(tmp, x=feature, y="Приложение", 
        title='Количество сделок в обучающей выборке {} дефолта'.format(type),
        labels=labels,
        color = 'Метка дефолта',
        text="Приложение", 
        # barmode='group'
        )

    config = dict({'modeBarButtonsToAdd': ['hovercompare']})
    fig.update_layout(autosize=False, width=800, height=400,)
    
    if x_range: 
        fig.update_xaxes(range=x_range)
    if y_range: 
        fig.update_yaxes(range=y_range)
    
    if show:
        fig.show('png' if need_svg else '', config=config)

    return fig

## prescoring.ipynb

In [1]:
# графики FP-TP для prescoring.ipynb
def show_depend_metrics(metrics, x, y, **kwargs):

    boundary = kwargs.get('LG_boundary', None)
    title = kwargs.get('title', x + " versus " + y)

    fig = px.line(metrics, x=x, y=y, markers=True, hover_data={'boundary':True, 'FN':True, 'TN':True, 'FP':True, 'TP':True}, text = 'boundary_notation')
    fig.update_traces(textposition='top center')

    if boundary:
        tmp = metrics[metrics['boundary']==boundary]
        x_annot = tmp[x].values[0]
        y_annot = tmp[y].values[0]
        fig.add_annotation(x=x_annot, y=y_annot, text="Log-regression result", showarrow=True, arrowhead=1)
    fig.update_layout(title_text=title)

    config = dict({'modeBarButtonsToAdd': ['hovercompare']})
    fig.update_layout(autosize=False, width=600, height=400,)
    fig.show('png' if need_svg else '', config=config)
    return fig


In [ ]:
# графики Количество контрагентов для pre-scoring при различных FN
def show_clients_versus_metric(df, **kwargs):
    
    x = kwargs.get('X', 'FN')
    x_lable = kwargs.get('x_lable', 'False negative')
    title = kwargs.get('title', "Количество контрагентов для pre-scoring в зависимости от вероятности дефолта")

    order = ['Есть действующий договор(ы)', 'Клиент в прошлом', 'Клиент в прошлом (расторжение)', 
            'Отказ УОБ', 'Черный список', 'Метка дефолта', 'Не было передачи в лизинг','Не является нашим клиентом']

    fig = px.bar(
        df, x=x,  y='Количество ИНН', 
        title=title, 
        color = 'Статус_общ', 
        barmode='stack',
        text_auto=True, 
        category_orders ={'Статус_общ':order}, 
        color_discrete_sequence=px.colors.qualitative.G10, 
        hover_data={'boundary':True},

    )
    #fig.update_layout(bargap=0.5)
    fig.update_yaxes(title_text="Количество контрагентов")
    # fig.update_traces(width = 0.3)
    fig.update_xaxes(title_text=x_lable)

    fig.update_layout(autosize=False, width=700, height=400,)
    fig.update_layout(modebar_add=['hovercompare'])

    return fig

In [ ]:
# графики Количество контрагентов для pre-scoring при различных FN
def show_clients_boundary_fix(df, **kwargs):
    
    # annot_boundary = kwargs.get('annot_boundary', None)
    title = kwargs.get('title', "Количество контрагентов при количестве договоров от 5 и более")

    order = ['Есть действующий договор(ы)', 'Клиент в прошлом', 'Клиент в прошлом (расторжение)', 
            'Отказ УОБ', 'Черный список', 'Метка дефолта', 'Не было передачи в лизинг','Не является нашим клиентом']

    fig = px.histogram(
        df, x='Статус_общ',  y='new_lessee_inn', 
        title=title, 
        barmode='stack',
        histfunc='count',
        text_auto=True,
        color = 'is_in_bmd', 
        category_orders ={'Статус_общ':order}, 
    )
    fig.update_xaxes(title_text="Статус контрагентов")

    fig.update_yaxes(title_text='Количество ИНН')

    config = dict({'modeBarButtonsToAdd': ['hovercompare']})
    fig.update_layout(autosize=False, width=700, height=400,)
    fig.show('png' if need_svg else '', config=config)

    return fig

In [ ]:
# графики Количество контрагентов для pre-scoring при различных отсечениях для prescoring.ipynb
def show_clients_for_prescoring(df, color, **kwargs):
    
    annot_boundary = kwargs.get('annot_boundary', None)

    order = ['Есть действующий договор(ы)', 'Клиент в прошлом', 'Клиент в прошлом (расторжение)', 
            'Отказ УОБ', 'Черный список', 'Метка дефолта', 'Не было передачи в лизинг','Не является нашим клиентом']

    fig = px.bar(
            df, x='boundary',  y='Количество ИНН', 
            title="Количество контрагентов для pre-scoring при различных отсечениях", 
            color = color, 
            barmode='stack',
            text_auto=True, 
            category_orders ={'Статус_общ':order}, 
            color_discrete_sequence=px.colors.qualitative.G10
        )
    fig.update_yaxes(title_text="Количество контрагентов")

    fig.update_xaxes(title_text='Граница отсечения (договоров)')

    if annot_boundary:
        x_annot = annot_boundary
        y_annot = df[df['boundary']==annot_boundary]['Количество ИНН'].sum()
        fig.add_annotation(x=x_annot, y=y_annot, text="Log-regression result", showarrow=True, arrowhead=1, ax=60, ay=-30)

    config = dict({'modeBarButtonsToAdd': ['hovercompare']})
    fig.update_layout(autosize=False, width=700, height=400,)
    fig.show('png' if need_svg else '', config=config)

    return fig

In [ ]:
def show_model_multifactors(df, x_feature, y_feature, **kwargs):
    title = kwargs.get('title', 'Модель Logistic regression')
    color = kwargs.get('color', 'pred')
    color_discrete_sequence = kwargs.get('color_discrete_sequence', ["red", "blue"])
    category_orders = kwargs.get('category_orders', {'y_pred':['0', '1']})
    
    x_range = kwargs.get('x_range', None)
    y_range = kwargs.get('y_range', None)
    
     
    fig = px.scatter(df, x=x_feature, y=y_feature,  color=color, 
                    color_discrete_sequence=color_discrete_sequence, 
                    labels={"pred": "Дефолт", 'proba':'Вероятность дефолта'},
                    category_orders = category_orders)
    
    fig.update_layout(title=title)
    fig.update_xaxes(title_text=get_feature_name(x_feature, 'title'))
    fig.update_yaxes(title_text=get_feature_name(y_feature, 'title'))
    
    if x_range: 
        fig.update_xaxes(range=x_range)
    if y_range: 
        fig.update_yaxes(range=y_range)
        
    config = dict({'modeBarButtonsToAdd': ['hovercompare']})
    fig.update_layout(autosize=False, width=700, height=400,)
    
    fig.show('png' if need_svg else '', config=config)

    return fig

In [ ]:
def show_model_multifactors_3d(df, **kwargs):
    title = kwargs.get('title', 'Модель Logistic regression')
    z = kwargs.get('z', 'pred')
    color = kwargs.get('color', 'proba')

    fig = px.scatter_3d(df, x='no_contracts_total', y='duration_max',   z=z, color=color, 
                        color_discrete_sequence=["red", "blue"], labels={"pred": "Прогнозный дефолт", 'proba':'Вероятность дефолта'})
    fig.update_layout(scene = dict(xaxis = dict(range=[0, 50],),
                                yaxis = dict( range=[0,500]), 
                                xaxis_title='Кол-во договоров',
                                yaxis_title='Макс. продолжительность',
                               
                                ),
                    )
    if z=='proba':
        fig.update_layout(scene = dict(zaxis = dict( range=[0.05,0.3]), 
                                    zaxis_title='Вероятность дефолта' ))
    elif z=='Количество ИНН':
        fig.update_layout(scene = dict(zaxis_type='log'))                              

    fig.update_layout(title=title)


    config = dict({'modeBarButtonsToAdd': ['hovercompare']})
    fig.update_layout(autosize=False, width=900, height=550,)
    fig.show('png' if need_svg else '', config=config)

    return fig

In [ ]:
def show_share_target(df, **kwargs):
    title = kwargs.get('title', 'Доля дефолта в target, %')
    x_range = kwargs.get('x_range', [0.1, 0.16])
    y_range = kwargs.get('y_range', [0, 9])

    fig = px.scatter(df, x='proba', y="Доля_дефолта_в_target_proc")
    fig.update_yaxes(title_text="Доля дефолта, %")
    fig.update_xaxes(title_text="Рассчитанная proba (вероятность дефолта) в log regression")
    
    if x_range: 
        fig.update_xaxes(range=x_range)
    if y_range: 
        fig.update_yaxes(range=y_range)
        
    fig.update_layout(title=title)

    config = dict({'modeBarButtonsToAdd': ['hovercompare']})
    fig.update_layout(autosize=False, width=700, height=400,)
    fig.show('png' if need_svg else '', config=config)
    
    return fig

In [ ]:
# графики Количество контрагентов для pre-scoring при различных отсечениях для prescoring.ipynb
def show_clients_for_prescoring_proba(df, color, **kwargs):
    
    annot_boundary = kwargs.get('annot_boundary', None)

    order = ['Есть действующий договор(ы)', 'Клиент в прошлом', 'Клиент в прошлом (расторжение)', 
            'Не было передачи в лизинг','Не является нашим клиентом', 
            'Отказ УОБ', 'Черный список', 'Метка дефолта', 
            'Недействующая организация', 'Четвёртая группа надежности',]

    fig = px.bar(
            df, x='proba',  y='Количество ИНН', 
            title="Количество контрагентов для pre-scoring при различных proba", 
            color = color, 
            barmode='stack',
            text_auto=True, 
            category_orders ={'Статус_общ':order}, 
            color_discrete_sequence=px.colors.qualitative.G10, 
            labels={"share_of_fraud": "Доля дефолта в target, %"}, 
            hover_data={'share_of_fraud':True},
        )
    fig.update_yaxes(title_text="Количество контрагентов")

    fig.update_xaxes(title_text='Рассчитанная proba (вероятность дефолта) в log regression')

    if annot_boundary:
        x_annot = annot_boundary
        y_annot = df[df['proba']==annot_boundary]['Количество ИНН'].sum()
        fig.add_annotation(x=x_annot, y=y_annot, text="2% дефолта в пуле", showarrow=True, arrowhead=1, ax=-60, ay=-30)

    config = dict({'modeBarButtonsToAdd': ['hovercompare']})
    fig.update_layout(autosize=False, width=800, height=500,)
    fig.show('png' if need_svg else '', config=config)

    return fig

## exploratory_been_denied_part2.ipynb

In [ ]:
# графики Количество контрагентов для pre-scoring при различных FN
def show_clients_denied(df, **kwargs):
    
    # annot_boundary = kwargs.get('annot_boundary', None)
    title = kwargs.get('title', 'Количество контрагентов среди отказников')

    order = ['Есть действующий договор(ы)', 'Клиент в прошлом', 'Клиент в прошлом (расторжение)', 
            'Отказ УОБ', 'Черный список', 'Метка дефолта', 'Не было передачи в лизинг','Не является нашим клиентом']

    fig = px.histogram(
        df, x='Статус_общ',  y='ИНН контрагента', 
        title=title, 
        barmode='stack',
        histfunc='count',
        text_auto=True,
        color = 'fedres', 
        category_orders ={'Статус_общ':order}, 
    )
    fig.update_xaxes(title_text="Статус контрагентов")

    fig.update_yaxes(title_text='Количество ИНН')

    config = dict({'modeBarButtonsToAdd': ['hovercompare']})
    fig.update_layout(autosize=False, width=700, height=400,)
    fig.show('png' if need_svg else '', config=config)
    
    return fig

## analysis_features

In [ ]:
def show_duration(df, x,  **kwargs):
    
    title = kwargs.get('title', 'Количество сделок от максимальной длительности закрытых договоров')
    show = kwargs.get('show', True)

    if (is_timedelta64_dtype(df[x])):
        df['duration_days'] = df[x].dt.days
    else:
        df['duration_days'] = df[x]

    fig = px.histogram(df, 
                       x='duration_days', 
                       y='index', 
                       title=title,
                       color='Метка дефолта', 
                       barmode='stack', 
                       histfunc='count') # , nbins= 10

    fig.update_xaxes(title_text="Дней")

    fig.update_yaxes(title_text='Количество сделок')

    config = dict({'modeBarButtonsToAdd': ['hovercompare']})
    fig.update_layout(autosize=False, width=700, height=400,)
    
    if show:
        fig.show('png' if need_svg else '', config=config)
    
    return fig

In [ ]:
def scatter_matrix(df, dimensions,  **kwargs):
    
    title = kwargs.get('title', 'scatter matrix')
    height = kwargs.get('height', 800)

    fig = px.scatter_matrix(df, dimensions=dimensions, color='Метка дефолта')
    fig.update_traces(diagonal_visible=False)
    fig.update_layout(title=title)
    fig.update_layout(autosize=False, width=height, height=height,)
    config = dict({'modeBarButtonsToAdd': ['hovercompare']})
    # fig.update_layout(margin = dict(t = 200)) заголовок опустить
    fig.show('png' if need_svg else '', config=config)
    
    return fig

## feature_selection

In [2]:
def correlation_matrix_plot(correlation_matrix, **kwargs):
    
    title = kwargs.get('title', 'Матрица корреляции')
    height = kwargs.get('height', 800)
    width = kwargs.get('width', 900)

    fig = px.imshow(correlation_matrix, text_auto=True)
    # fig.update_traces(dict(showscale=False, coloraxis=None))
    
    fig.update_layout(coloraxis_showscale=False)
    fig.update_layout(title=title)
    fig.update_layout(autosize=False, width=width, height=height,)
    config = dict({'modeBarButtonsToAdd': ['hovercompare']})
    fig.show('png' if need_svg else '', config=config)
    
    return fig

## feature_distribution_stability

In [ ]:
def plot_statistics_over_time(df, feature, type='фрод', **kwargs):
    x_range = kwargs.get('x_range', None)
    y_range = kwargs.get('y_range', None)
    title = kwargs.get('title', 'Изменение распределения фичи во времени')


    # df['error_y_minus'] = np.where(df['mean']>df['std'], df['std'], df['mean'])

    fig = px.scatter(df, x='time', y=['mean'], error_y="std", error_y_minus="error_y_minus")

    # fig = px.scatter(df, x='time', y=['mean'], error_y="std", error_y_minus="error_y_minus", trendline="ols", trendline_color_override='red')
    # fig = px.scatter(df, x='time', y=['mean'], error_y="std", error_y_minus="error_y_minus", 
    #                 trendline="rolling", trendline_options=dict(window=3), trendline_color_override='red')
    fig.update_traces(marker=dict(size=12))

    fig.add_trace(go.Scatter(x=df['time'], y=df['95%'], name = '95%',  mode="markers"))
    fig.add_trace(go.Scatter(x=df['time'], y=df['max'], name = 'max',  mode="lines"))


    config = dict({'modeBarButtonsToAdd': ['hovercompare']})
    fig.update_layout(autosize=False, width=800, height=500,)
    
    if x_range: 
        fig.update_xaxes(range=x_range)
    if y_range: 
        fig.update_yaxes(range=y_range)

    fig.update_layout(title=title)
    fig.update_yaxes(title_text=get_feature_name(feature, 'title'))
    # fig.show(config=config)

    return fig

In [1]:
def plot_statistics_over_time_mdm_target(df_target, df_mdm, **kwargs):
    x_range = kwargs.get('x_range', None)
    y_range = kwargs.get('y_range', None)
    title = kwargs.get('title', 'Изменение распределения фичи во времени')

    fig  = make_subplots(rows = 1, cols=2, subplot_titles = ['target', 'MDM'])
    fig.append_trace(go.Scatter(x=df_target['time'], y=df_target['mean'], 
                    error_y=dict(type='data', array=df_target['std'], arrayminus =df_target['error_y_minus'],  visible=True, color='blue'),
                        mode="markers", name='mean',  
                        marker=dict(color='blue', size=12)), row = 1, col = 1)

    fig.add_trace(go.Scatter(x=df_target['time'], y=df_target['95%'], name = '95%',  mode="markers", marker=dict(color='red')), row = 1, col = 1)
    fig.add_trace(go.Scatter(x=df_target['time'], y=df_target['max'], name = 'max',  mode="lines", marker=dict(color='green')), row = 1, col = 1)


    fig.append_trace(go.Scatter(x=df_mdm['time'], y=df_mdm['mean'], mode="markers", name='mean',  
                        error_y=dict(type='data', array=df_mdm['std'], arrayminus =df_mdm['error_y_minus'],  visible=True, color='blue'),
                        marker=dict(color='blue', size=12), showlegend=False), row = 1, col = 2)
    
    fig.add_trace(go.Scatter(x=df_mdm['time'], y=df_mdm['95%'], name = '95%',  mode="markers", showlegend=False, marker=dict(color='red')), row = 1, col = 2)
    fig.add_trace(go.Scatter(x=df_mdm['time'], y=df_mdm['max'], name = 'max',  mode="lines", showlegend=False, marker=dict(color='green')), row = 1, col = 2)


    if y_range: 
        fig.update_yaxes(range=y_range)

    fig.update_layout(autosize=False, width=1600, height=500,)
    fig.update_layout(title=title, font=dict(size=18,))
    return fig

In [ ]:
def plot_qqplot(df_pct, name_theor, name_sample, **kwargs):
    x_range = kwargs.get('x_range', None)
    y_range = kwargs.get('y_range', None)
    title = kwargs.get('title', 'qq-plot')
    x_title = kwargs.get('x_title', 'Theoretical Quantiles')
    y_title = kwargs.get('y_title', 'Sample Quantiles')

    fig = go.Figure(go.Scatter(x=df_pct[name_theor], y=df_pct[name_theor], name = name_theor,  mode="lines+markers", 
                line=dict(color='red'), 
                text =df_pct['percentile'], hovertemplate=
                                    "<b>"+feature+"</b><br>" +
                                    "<b>percentile %{text}</b><br>" +
                                     name_theor + ": %{x}<br>" +
                                    "<extra></extra>",
                ))

    fig.add_trace(go.Scatter(x=df_pct[name_theor], y=df_pct[name_sample], name = name_sample,  mode="markers", marker=dict(color='blue'), 
                text =df_pct['percentile'], hovertemplate=
                                    "<b>"+feature+"</b><br>" +
                                    "<b>percentile %{text}</b><br>" +
                                      name_theor + ": %{x}<br>" +
                                    name_sample + ": %{y}<br>" +
                                    "<extra></extra>",
                                    ))
   
 
    fig.update_xaxes(title_text=x_title)
    fig.update_yaxes(title_text=y_title)
    fig.update_layout(title=title)
        
    config = dict({'modeBarButtonsToAdd': ['hovercompare']})
    fig.update_layout(autosize=False, width=600, height=600,)
    # fig.show(config=config)
    return fig

In [ ]:
def plot_qqplot_in_subplots(df_pct, month_1, month_2, **kwargs):
    x_range = kwargs.get('x_range', None)
    y_range = kwargs.get('y_range', None)

    fig  = make_subplots(rows = 1, cols=3, subplot_titles = ['target', 'дефолты in target', 'не дефолты in target'])
    
    showlegend = True
    for col, type in enumerate(['overall', 'defaults', 'not_defaults'], start=1):
        fig.append_trace(go.Scatter(x=df_pct[type+month_1], y=df_pct[type+month_1], name = month_1,  mode="lines+markers", 
                    line=dict(color='red'), 
                    text =df_pct['percentile'], hovertemplate=
                                        "<b>"+feature+"</b><br>" +
                                        "<b>percentile %{text}</b><br>" +
                                        "эталон: %{x}<br>" +
                                        "<extra></extra>",
                    showlegend=showlegend), row = 1, col = col)

        fig.add_trace(go.Scatter(x=df_pct[type+month_1], y=df_pct[type+month_2], name = month_2,  mode="markers", marker=dict(color='blue'), 
                    text =df_pct['percentile'], hovertemplate=
                                        "<b>"+feature+"</b><br>" +
                                        "<b>percentile %{text}</b><br>" +
                                        "month эталон: %{x}<br>" +
                                        "month test: %{y}<br>" +
                                        "<extra></extra>",
                                        showlegend=showlegend), row = 1, col = col)
        showlegend=False

    title = 'Q-Q plot фичи <b>{}</b>'.format(feature)+ '<br>'+'в 2 моментах времени {} и {}'.format(month_1, month_2)
    fig.update_layout(autosize=False, width=1600, height=500,)
    fig.update_layout(title=title, font=dict(size=18,))
        
    config = dict({'modeBarButtonsToAdd': ['hovercompare']})
    # fig.show(config=config)
    return fig

In [ ]:
def q_q_histogram(df_pct, df_target, df_mdm, feature, name_theor, name_sample, **kwargs):
    x_range = kwargs.get('x_range', None)
    y_range = kwargs.get('y_range', None)
    title = kwargs.get('title', 'qq-plot')
    x_title = kwargs.get('x_title', 'Theoretical Quantiles')
    y_title = kwargs.get('y_title', 'Sample Quantiles')
    log_scale = kwargs.get('log_scale', False)
    feature_name = kwargs.get('feature_name', 'feature')

    fig  = make_subplots(rows = 1, cols=3, subplot_titles = ['q-q plot', 'target', 'MDM'])
    fig.add_trace(go.Scatter(x=df_pct[name_theor], y=df_pct[name_theor], mode="lines+markers", line=dict(color='green'),
                text =df_pct['percentile'], hovertemplate=
                                    "<b>"+feature+"</b><br>" +
                                    "<b>percentile %{text}</b><br>" +
                                     name_theor + ": %{x}<br>" +
                                    "<extra></extra>",
                legendgroup="qq",  legendgrouptitle_text="q-q plot", name = name_theor, legendrank=2,
                ), row = 1, col = 1)

    fig.add_trace(go.Scatter(x=df_pct[name_theor], y=df_pct[name_sample],   mode="markers", marker=dict(color='MediumPurple'), 
                text =df_pct['percentile'], hovertemplate=
                                    "<b>"+feature+"</b><br>" +
                                    "<b>percentile %{text}</b><br>" +
                                      name_theor + ": %{x}<br>" +
                                    name_sample + ": %{y}<br>" +
                                    "<extra></extra>",
                legendgroup="qq",  legendgrouptitle_text="q-q plot", name = name_sample, legendrank=1,
                 ), row = 1, col = 1)
   

    fig.update_layout(xaxis=dict(title_text=x_title))
    fig.update_layout(yaxis=dict(title_text=y_title))

    hist_target = px.histogram(
                df_target, x=feature,  y='Приложение', 
                color = 'Метка дефолта', 
                barmode='stack',
                histfunc='count'
                )
    hist_mdm = px.histogram(df_mdm, x=feature,  y='new_lessee_inn', histfunc='count')

    fig.add_trace(go.Histogram(x=hist_target.data[0].x, y=hist_target.data[0].y, marker=dict(color='blue'),
                    # legendgroup="target",  legendgrouptitle_text="target распределение", 
                    name = 'Метка дефолта = 0', legendrank=4
                    ), row = 1, col = 2)
    fig.add_trace(go.Histogram(x=hist_target.data[1].x, y=hist_target.data[1].y,  marker=dict(color='red'),
                # legendgroup="target",  legendgrouptitle_text="target распределение", 
                name = 'Метка дефолта = 1', legendrank=5
                ), row = 1, col = 2)
    fig.update_layout(barmode="stack",    bargap=0.2,   bargroupgap=0.1)
    if log_scale:
        fig.update_layout(yaxis2=dict(type='log'))
    fig.update_layout(xaxis2=dict(range=x_range))
    
    fig.add_trace(go.Histogram(x=hist_mdm.data[0].x, y=hist_mdm.data[0].y, marker=dict(color='green'),
                    legendgroup="mdm",  legendgrouptitle_text="MDM распределение", name = 'фед ресурс',  
                    ), row = 1, col = 3)
    if log_scale:
        fig.update_layout(yaxis3=dict(type='log'))
    
    fig.update_layout(xaxis3=dict(title_text=feature_name, range=x_range))

    fig.update_layout(autosize=False, width=1500, height=500,)
    fig.update_layout(title=title, font=dict(size=18,))
    return fig

def q_q_bar(df_pct, df_target, df_mdm, feature, name_theor, name_sample, **kwargs):
    x_range = kwargs.get('x_range', None)
    y_range = kwargs.get('y_range', None)
    title = kwargs.get('title', 'qq-plot')
    x_title = kwargs.get('x_title', 'Theoretical Quantiles')
    y_title = kwargs.get('y_title', 'Sample Quantiles')
    log_scale = kwargs.get('log_scale', False)
    feature_name = kwargs.get('feature_name', 'feature')

    fig  = make_subplots(rows = 1, cols=3, subplot_titles = ['q-q plot', 'target', 'MDM'])
    fig.add_trace(go.Scatter(x=df_pct[name_theor], y=df_pct[name_theor], mode="lines+markers", line=dict(color='green'),
                text =df_pct['percentile'], hovertemplate=
                                    "<b>"+feature+"</b><br>" +
                                    "<b>percentile %{text}</b><br>" +
                                     name_theor + ": %{x}<br>" +
                                    "<extra></extra>",
                legendgroup="qq",  legendgrouptitle_text="q-q plot", name = name_theor, legendrank=2,
                ), row = 1, col = 1)

    fig.add_trace(go.Scatter(x=df_pct[name_theor], y=df_pct[name_sample],   mode="markers", marker=dict(color='MediumPurple'), 
                text =df_pct['percentile'], hovertemplate=
                                    "<b>"+feature+"</b><br>" +
                                    "<b>percentile %{text}</b><br>" +
                                      name_theor + ": %{x}<br>" +
                                    name_sample + ": %{y}<br>" +
                                    "<extra></extra>",
                legendgroup="qq",  legendgrouptitle_text="q-q plot", name = name_sample, legendrank=1,
                 ), row = 1, col = 1)
   

    fig.update_layout(xaxis=dict(title_text=x_title))
    fig.update_layout(yaxis=dict(title_text=y_title))

    target_distr = pd.pivot_table(df_target, index=[feature, 'Метка дефолта'], values='Приложение', aggfunc='count', fill_value=0)\
                    .reset_index()
    mask = (target_distr['Метка дефолта']==0)
    target_distr_0 = target_distr[mask]
    target_distr_1 = target_distr[~mask]

    mdm_distr = pd.pivot_table(df_mdm, index=feature, values='new_lessee_inn', aggfunc='nunique', fill_value=0).reset_index()

    fig.add_trace(go.Bar(x=target_distr_0[feature], y=target_distr_0["Приложение"],  marker=dict(color='blue'), 
                # legendgroup="target",  legendgrouptitle_text="target распределение", 
                name = 'Метка дефолта = 0', legendrank=4
                ), row = 1, col = 2)
    fig.add_trace(go.Bar(x=target_distr_1[feature], y=target_distr_1["Приложение"], marker=dict(color='red'),
                # legendgroup="target",  legendgrouptitle_text="target распределение", 
                name = 'Метка дефолта = 1', legendrank=5
                ), row = 1, col = 2)
    fig.update_layout(barmode="stack")

    fig.update_layout(xaxis2=dict(range=x_range))
    # fig.update_layout(yaxis2=dict(title_text='Количество сделок'))

    fig.add_trace(go.Bar(x=mdm_distr[feature], y=mdm_distr["new_lessee_inn"],  marker=dict(color='green'),
                legendgroup="mdm",  legendgrouptitle_text="MDM распределение", name = 'фед ресурс', 
                ), row = 1, col = 3)

    fig.update_layout(xaxis3=dict(title_text=feature_name, range=x_range))

    fig.update_layout(autosize=False, width=1500, height=500,)
    fig.update_layout(title=title, font=dict(size=18,))
    return fig
  


In [ ]:
def ecdf_plot(x_target, count_target, x_mdm, count_mdm, feature, **kwargs):
    x_range = kwargs.get('x_range', None)
    y_range = kwargs.get('y_range', None)
    title = kwargs.get('title', 'ecdf')
    x_title = kwargs.get('x_title', feature)
    y_title = kwargs.get('y_title', 'F(x)')
    feature_name = kwargs.get('feature_name', 'feature')

    fig = go.Figure()
    fig.add_traces(go.Scatter(x=x_target, y = count_target,  mode="lines", 
            line=dict(color='green'), name = 'target', 
             hovertemplate="<b>"+feature_name+"</b><br>" +
                            "<b>target</b><br>" +
                            feature +" %{x}<br>" +
                            "ecdf %{y}<br>" +
                            "<extra></extra>",  
                    ))
    fig.add_traces(go.Scatter(x=x_mdm, y = count_mdm,   mode="lines", 
            line=dict(color='violet'), name = 'mdm',
            hovertemplate="<b>"+feature_name+"</b><br>" +
                            "<b>mdm</b><br>" +
                            feature +" %{x}<br>" +
                            "ecdf %{y}<br>" +
                            "<extra></extra>",  ))
   
    fig.update_xaxes(range=x_range, title_text=feature_name)
    fig.update_yaxes(title_text=y_title)
    fig.update_layout(autosize=False, width=800, height=500,)
    fig.update_layout(title=title, font=dict(size=18,))
    return fig

#  еще вариант
# hist_target = px.histogram(df_target, x=feature,  y='Приложение',           
#             histfunc='count')
# fig = px.ecdf(df_target, x=feature, color='Метка дефолта', markers=True, lines=True)

In [ ]:
def psi_plot(df, **kwargs):
    x_range = kwargs.get('x_range', None)
    y_range = kwargs.get('y_range', None)
    title = kwargs.get('title', 'psi')
    x_title = kwargs.get('x_title', 'Месяц')
    y_title = kwargs.get('y_title', 'psi')
    feature = kwargs.get('feature', 'feature')
    feature_name = kwargs.get('feature_name', 'feature')

    fig = go.Figure()
    fig.add_traces(go.Scatter(x=df['month'], y = df['psi_value'],  mode="lines", 
            line=dict(color='green'), name ='psi of '+feature, 
            hovertemplate="<b>"+feature_name+"</b><br>" +
                            "month" +" %{x}<br>" +
                            "psi %{y}<br>" +
                            "<extra></extra>",  
                    ))

    x0 = df['month'].min()
    x1 = df['month'].max()
    fig.add_shape(type='line', line=dict(dash='dash',color='red'),
            x0=x0, x1=x1, y0=0.1, y1=0.1, name='stability boundary')
        
    fig.update_xaxes(title_text=x_title)
    fig.update_yaxes(title_text=y_title)
    fig.update_layout(autosize=False, width=800, height=500,)
    fig.update_layout(title=title, font=dict(size=18,))
    return fig

In [ ]:
def plot_statistics_over_time_target(df, feature, type='фрод', **kwargs):
    x_range = kwargs.get('x_range', None)
    y_range = kwargs.get('y_range', None)
    title = kwargs.get('title', 'Изменение распределения фичи во времени')


    fig = px.scatter(df, x='time', y=['mean'])

    fig.update_traces(marker=dict(size=12))

    fig.add_trace(go.Scatter(x=df['time'], y=df['95%'], name = '95%',  mode="markers"))
    fig.add_trace(go.Scatter(x=df['time'], y=df['median'], name = 'median',  mode="markers"))


    config = dict({'modeBarButtonsToAdd': ['hovercompare']})
    fig.update_layout(autosize=False, width=800, height=500,)
    
    if x_range: 
        fig.update_xaxes(range=x_range)
    if y_range: 
        fig.update_yaxes(range=y_range)

    fig.update_layout(title=title)
    fig.update_yaxes(title_text=get_feature_name(feature, 'title'))
    # fig.show(config=config)

    return fig

In [ ]:
def plot_feature_statistics_over_time_mdm_target(df_target, df_mdm, **kwargs):
    x_range = kwargs.get('x_range', None)
    y_range = kwargs.get('y_range', None)
    title = kwargs.get('title', 'Изменение распределения фичи во времени')

    fig  = make_subplots(rows = 1, cols=2, subplot_titles = ['Обучающая выборка', 'MDM'])
    fig.append_trace(go.Scatter(x=df_target['time'], y=df_target['mean'], 
                        mode="markers", name='mean',  
                        marker=dict(color='blue', size=12)), row = 1, col = 1)

    fig.add_trace(go.Scatter(x=df_target['time'], y=df_target['95%'], name = '95%',  mode="markers", marker=dict(color='red')), row = 1, col = 1)
    fig.add_trace(go.Scatter(x=df_target['time'], y=df_target['median'], name = 'median',  mode="markers", marker=dict(color='green')), row = 1, col = 1)


    fig.append_trace(go.Scatter(x=df_mdm['time'], y=df_mdm['mean'], mode="markers", name='mean',  
                       marker=dict(color='blue', size=12), showlegend=False), row = 1, col = 2)
    
    fig.add_trace(go.Scatter(x=df_mdm['time'], y=df_mdm['95%'], name = '95%',  mode="markers", showlegend=False, marker=dict(color='red')), row = 1, col = 2)
    fig.add_trace(go.Scatter(x=df_mdm['time'], y=df_mdm['median'], name = 'median',  mode="markers", showlegend=False, marker=dict(color='green')), row = 1, col = 2)


    if y_range: 
        fig.update_yaxes(range=y_range)

    fig.update_layout(autosize=False, width=1200, height=500,)
    fig.update_layout(title=title, font=dict(size=18,))
    return fig

In [ ]:
def psi_plot_2(df, feature, base_periods,  **kwargs):
    x_range = kwargs.get('x_range', None)
    y_range = kwargs.get('y_range', None)
    title = kwargs.get('title', 'psi')
    x_title = kwargs.get('x_title', 'Месяц')
    y_title = kwargs.get('y_title', 'psi')
    feature_name = kwargs.get('feature_name', 'feature')
    
    
    fig  = make_subplots(rows = 1, cols=2, subplot_titles = ["base period: {}".format(base_periods[0]), "base period: {}".format(base_periods[1])])
    
    for col, period in enumerate(base_periods):
        mask = (df['base_period']==period)
        df_part= df[mask]

        fig.append_trace(go.Scatter(x=df_part['month'], y = df_part['psi_value'],  mode="lines", 
                line=dict(color='green'), name ='base period '+ period, 
                hovertemplate="<b>"+feature_name+"</b><br>" +
                                "month" +" %{x}<br>" +
                                "psi %{y}<br>" +
                                "<extra></extra>",  
                        showlegend=False), row = 1, col = col+1, )

        x0 = df_part['month'].min()
        x1 = df_part['month'].max()
        fig.add_shape(type='line', line=dict(dash='dash',color='red'),
                x0=x0, x1=x1, y0=0.1, y1=0.1, name='stability boundary', row = 1, col = col+1)

     
    fig.update_xaxes(title_text=x_title)
#     fig.update_yaxes(title_text=y_title)
    fig.update_layout(autosize=False, width=1200, height=500,)
    fig.update_layout(title=title, font=dict(size=18,))
    return fig

In [ ]:
def plot_psi_features(df, **kwargs):
    x_range = kwargs.get('x_range', None)
    y_range = kwargs.get('y_range', None)
    title = kwargs.get('title', 'PSI target-class versus mdm distributions')

    fig = px.scatter(df, x='feature', y=['psi_value']  )

    # fig.update_traces(marker=dict(size=12))
    df['boundary'] = 0.1
    df['boundary_2'] = 0.25

    fig.add_trace(go.Scatter(x=df['feature'], y=df['boundary'], name = 'граница стабильности',  
                mode="lines", line=dict(dash='dash',color='green')))

    fig.add_trace(go.Scatter(x=df['feature'], y=df['boundary_2'], name = 'граница нестабильности', 
                 mode="lines", line=dict(dash='dash',color='red')))

    config = dict({'modeBarButtonsToAdd': ['hovercompare']})
    fig.update_layout(autosize=False, width=1000, height=600,)
    
    if x_range: 
        fig.update_xaxes(range=x_range)
    if y_range: 
        fig.update_yaxes(range=y_range)
    
    fig.update_xaxes(tickangle=-45,  title_text='')
    fig.update_yaxes(title_text='')
    fig.update_layout(title=title)
    fig.show('png' if need_svg else '', config=config)

    return fig

## feature_one_factor_analysis

In [ ]:
def plot_metrics(df, **kwargs):
    x_range = kwargs.get('x_range', None)
    y_range = kwargs.get('y_range', None)
    title = kwargs.get('title', 'metrics в обучающей выборке при однофакторном анализе')

    fig = px.scatter(df, x='feature', y=['value'], color='metric' )
    
    df['boundary'] = 0.6

    fig.add_trace(go.Scatter(x=df['feature'], y=df['boundary'], name = 'граница 0.6 (для rocauc)',  
                mode="lines", line=dict(dash='dash',color='green')))
  
    config = dict({'modeBarButtonsToAdd': ['hovercompare']})
    fig.update_layout(autosize=False, width=1000, height=600,)
    
    if x_range: 
        fig.update_xaxes(range=x_range)
    if y_range: 
        fig.update_yaxes(range=y_range)
    
    fig.update_xaxes(tickangle=-45,  title_text='')
    fig.update_yaxes(title_text='')
    fig.update_layout(title=title)
    fig.show('png' if need_svg else '', config=config)

    return fig

## mvp_model.ipynb

In [ ]:
# графики Количество pred c плохим завершением
def cpPredictedBadCases(*args, **kwargs):
    
    df = kwargs.get('df', None)
    
    distr = pd.pivot_table(df, index=['y_pred', 'Метка дефолта'], values='Приложение', aggfunc='count', fill_value=0).reset_index()

    fig = px.bar(
        distr, x='y_pred',  y='Приложение', 
        color = 'Метка дефолта', 
        barmode='stack',
        text_auto=True, 
    )
    
    return fig

In [ ]:
# графики Количество pred c плохим завершением
def show_pred(df, **kwargs):
    x_range = kwargs.get('x_range', None)
    y_range = kwargs.get('y_range', None)
    
    title = kwargs.get('title', "Predicted values сделок с плохим завершением")

    distr = pd.pivot_table(df, index=['y_pred', 'Метка дефолта'], values='Приложение', aggfunc='count', fill_value=0).reset_index()

    fig = px.bar(
        distr, x='y_pred',  y='Приложение', 
        title=title, 
        color = 'Метка дефолта', 
        barmode='stack',
        text_auto=True, 
    )
    
    fig.update_yaxes(title_text="Количество сделок")

    fig.update_xaxes(title_text='Predicted values (предсказанные значения)')

    fig.update_layout(autosize=False, width=700, height=500,)
    fig.update_layout(modebar_add=['hovercompare'])

    return fig

In [1]:
# графики Количество pred c плохим завершением
def cpPairplot(*args, **kwargs):
    
    df = kwargs.get('df', None)
    x_feature = kwargs.get('x_feature', None)
    y_feature = kwargs.get('y_feature', None)
    color = kwargs.get('color', None)
    colorscale = kwargs.get('colorscale', 'RdBu_r')
    labels = kwargs.get('labels', None)
                
    # colorscale=[[0.0, "rgb(0, 0, 255)"],
    #             [0.05, "rgb(20, 0, 200)"],
    #             [0.1, "rgb(50, 0, 150)"],
    #             [0.15, "rgb(75, 0, 125)"],
    #             [0.20, "rgb(100, 0, 100)"],
    #             [0.30, "rgb(150, 0, 75)"],
    #             [0.40, "rgb(175, 0, 50)"],
    #             [0.50, "rgb(200, 0, 20)"],
    #             [1.0, "rgb(255, 0, 0)"]]
# 'RdBu_r'
    fig = px.scatter(df, x=x_feature, y=y_feature,  color=color, 
                    color_continuous_scale=colorscale, labels=labels)
    
    return fig

In [ ]:
def show_pairplot(df, x_feature, y_feature, **kwargs):
    title = kwargs.get('title', 'Pair-plot')
    color = kwargs.get('color', 'pred')
    x_range = kwargs.get('x_range', None)
    y_range = kwargs.get('y_range', None)

    fig = px.scatter(df, x=x_feature, y=y_feature,  color=color, 
                    color_discrete_sequence=[ "blue", "red"], labels={"pred": "Дефолт", 'proba':'Вероятность дефолта'})
    
    fig.update_layout(title=title)
    
    fig.update_xaxes(title_text=get_feature_name(x_feature, 'title'))
    fig.update_yaxes(title_text=get_feature_name(y_feature, 'title'))
    
    if x_range: 
        fig.update_xaxes(range=x_range)
    if y_range: 
        fig.update_yaxes(range=y_range)
        

    config = dict({'modeBarButtonsToAdd': ['hovercompare']})
    fig.update_layout(autosize=False, width=700, height=400,)
    
    fig.show('png' if need_svg else '', config=config)

    return fig

In [ ]:
#  потом написать в общем виде нужно соединение несколько готовых fig
def pair_subplots(fig_pairplot_frod, fig_pairplot_cred):
    fig  = make_subplots(rows = 1, cols=2, subplot_titles = ['фрод дефолт', 'кред дефолт'])


    fig.append_trace(fig_pairplot_frod['data'][0], 1, 1)
    fig.append_trace(fig_pairplot_frod['data'][1], 1, 1)

    fig_pairplot_cred['data'][0]['showlegend']= False
    fig_pairplot_cred['data'][1]['showlegend']= False

    fig.append_trace(fig_pairplot_cred['data'][0], 1, 2)
    fig.append_trace(fig_pairplot_cred['data'][1], 1, 2, )


    fig.update_layout(xaxis2=dict(range=fig_pairplot_cred['layout']['xaxis']['range']))
    fig.update_layout(yaxis2=dict(range=fig_pairplot_cred['layout']['yaxis']['range']))
    fig.update_layout(xaxis1=dict(range=fig_pairplot_frod['layout']['xaxis']['range']))

    fig.update_layout(xaxis1=dict(title=fig_pairplot_cred['layout']['xaxis']['title']))
    fig.update_layout(xaxis2=dict(title=fig_pairplot_cred['layout']['xaxis']['title']))

    fig.update_layout(yaxis1=dict(title=fig_pairplot_frod['layout']['yaxis']['title']))

    fig.update_layout(legend=dict(title=fig_pairplot_cred['layout']['legend']['title']))


    fig.update_layout(title=dict(text='pair-plot фичей модели MVP', font=dict(size=18,)))

    fig.show('png' if need_svg else '')
    
    return fig

In [ ]:
#  потом написать в общем виде нужно соединение несколько готовых fig
def pair_subplots2(fig_pairplot_frod, fig_pairplot_cred):
    fig  = make_subplots(rows = 1, cols=2, subplot_titles = ['фрод дефолт', 'кред дефолт'])


    fig.append_trace(fig_pairplot_frod['data'][0], 1, 1)

    fig_pairplot_cred['data'][0]['showlegend']= False


    fig.append_trace(fig_pairplot_cred['data'][0], 1, 2)


    fig.update_layout(xaxis2=dict(range=fig_pairplot_cred['layout']['xaxis']['range']))
    fig.update_layout(yaxis2=dict(range=fig_pairplot_cred['layout']['yaxis']['range']))
    fig.update_layout(xaxis1=dict(range=fig_pairplot_frod['layout']['xaxis']['range']))

    fig.update_layout(xaxis1=dict(title=fig_pairplot_cred['layout']['xaxis']['title']))
    fig.update_layout(xaxis2=dict(title=fig_pairplot_cred['layout']['xaxis']['title']))

    fig.update_layout(yaxis1=dict(title=fig_pairplot_frod['layout']['yaxis']['title']))

    fig.update_layout(legend=dict(title=fig_pairplot_cred['layout']['legend']['title']))


    fig.update_layout(title=dict(text='pair-plot фичей модели MVP', font=dict(size=18,)))

    fig.show('png' if need_svg else '')
    
    return fig

In [ ]:
def cpShareUzdlvTestDefaul(*args, **kwargs):
    df = kwargs.get('df', None)
    
    fig = px.scatter(df, x='Доля_дефолта_в_target_proc', y="Доля_УЗДЛ", hover_data={'proba':True})
    
    return fig

In [ ]:
#  написать потом в общем виде
def combine_figs(plotly_figs, rows = 1, cols=2):

    subplot_titles = kwargs.get('subplot_titles', ['фрод дефолт', 'кред дефолт'])
    
    fig_general  = make_subplots(rows = rows, cols=cols, subplot_titles = subplot_titles)
    row = 1
    col = 1
    num = 0
    for row in range(1, rows+1):
        for col in range(1, cols+1):
            fig  = plotly_figs[num]
            fig_general.append_trace(fig['data'][0], row, col)
            num += 1
            # fig_general.layout.update(fig.layout)
            
            # fig.update_layout(xaxis=dict(width=500))
    return fig_general

# combine_figs([fig_share_uzdl_cred, fig_share_uzdl_cred])     

In [ ]:
def cpPredictionsOnData(*args, **kwargs):
    df = kwargs.get('df', None)
    df_second = kwargs.get('df_second', None)
    x_feature = kwargs.get('x_feature', None)
    y_feature = kwargs.get('y_feature', None)
    color = kwargs.get('color', None)
    classes = kwargs.get('classes', ['дефолт=0', 'дефолт=1'])
    
    hist = px.histogram(
        df, x=x_feature,  y=y_feature, 
        color = 'Метка дефолта', 
        barmode='group',
        histfunc='count'
    )
    
    fig = make_subplots(specs=[[{"secondary_y": True}]])


    fig.add_trace(go.Histogram(x=hist.data[0].x, y=hist.data[0].y, name=classes[0], texttemplate="%{y}"),  secondary_y=False)
    fig.add_trace(go.Histogram(x=hist.data[1].x, y=hist.data[1].y, name=classes[1], texttemplate="%{y}"),  secondary_y=False)
    
    fig.add_trace(go.Scatter(x=df_second['X'], y=df_second['y_prob'], name='Вероятность дефолта', 
                                marker_color=df_second['y_pred'],
                                mode='markers', texttemplate = "%{y:.2f}", textposition = "top center" , 
                                marker=dict(
                                            size=10,
                                            colorscale=  [(0,"blue"), (1,"red")]
                                            )
                            ) , secondary_y=True)
    
    
    fig.update_yaxes(secondary_y=False, type='log')
    
    return fig

In [ ]:
# графики c sigmoid
def show_predictions(target, sigmoid_pred, feature, **kwargs):
    x_range = kwargs.get('x_range', None)
    y_range = kwargs.get('y_range', None)
    
    mask = kwargs.get('mask', None)
    if isinstance(mask, pd.Series):
        df = target.data[mask]
    else:
        df = target.data

    title = kwargs.get('title', feature)
    
    name_0 = "{type}=0".format(type=target.type)
    name_1 = "{type}=1".format(type=target.type)

    hist = px.histogram(
        df, x=feature,  y='Приложение', 
        # title="Количество активных договоров", 
        color = 'Метка дефолта', 
        barmode='group',
        histfunc='count'
    )
    
    default = sigmoid_pred[sigmoid_pred['y_pred']==1]
    not_default = sigmoid_pred[sigmoid_pred['y_pred']!=1]

    fig = make_subplots(specs=[[{"secondary_y": True}]])

  

    fig.add_trace(go.Histogram(x=hist.data[0].x, y=hist.data[0].y, name=name_0, texttemplate="%{y}"),  secondary_y=False)
    fig.add_trace(go.Histogram(x=hist.data[1].x, y=hist.data[1].y, name=name_1, texttemplate="%{y}"),  secondary_y=False)
    fig.add_trace(go.Scatter(x=default['X'], y=default['y_prob'], name='predicted 1', 
                                mode='markers', texttemplate = "%{y:.2f}", textposition = "top center" , 
                                marker=dict(
                                            size=10,
                                            color = 3, 
                                            colorscale='Viridis', # one of plotly colorscales
                                            )
                            ) , secondary_y=True)

    fig.add_trace(go.Scatter(x=not_default['X'], y=not_default['y_prob'], name='predicted 0', 
                                mode='markers', texttemplate = "%{y:.2f}", textposition = "top center" , 
                                marker=dict(
                                            size=10,
                                            color = 0, 
                                            colorscale='Bluered_r', # one of plotly colorscales
                                            )
                            ) , secondary_y=True)

    fig.update_layout(title_text="Количество target договоров и Вероятность дефолта predicted (balanced)".format(type=target.type))

    # Set y-axes titles
    fig.update_yaxes(title_text="Количество target договоров", secondary_y=False, type='log')
    # fig.update_yaxes(title_text="Количество target договоров", secondary_y=False, type='linear')
    fig.update_yaxes(title_text="Вероятность дефолта (balanced)".format(type=target.type), secondary_y=True)

    fig.update_xaxes(title_text=title)

    if x_range: 
        fig.update_xaxes(range=x_range)
    if y_range: 
        fig.update_yaxes(range=y_range)
        
    config = dict({'modeBarButtonsToAdd': ['hovercompare']})

    fig.update_layout(autosize=False, width=700, height=400,)
    fig.show('png' if need_svg else '', config=config)
    # fig.write_html("file.html")
    return fig